In [11]:
# from ruclip_dataset import RuCLIPDataset
# from ruclip import RuCLIPProcessor
import os
import pandas as pd
# import ruclip

In [1]:
import ruclip

d:\Programming\jupyterD\WORK\venv_second\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
!you --version

Python 3.12.2


In [5]:
import youtokentome

In [14]:
device = 'cpu'
clip, processor = ruclip.load('ruclip-vit-base-patch32-384', device=device)

In [ ]:
images_urls_names = pd.read_csv('/kaggle/input/indeepa-images/images_urls_names.csv')
images_urls_names.set_index('NmId', inplace=True)
images_urls_names.rename(columns={'name':'caption'}, inplace=True)

path = 'indeepa_images/2 parts/'
d_train, d_test = {}, {}
with os.scandir(path + 'train') as it:
    for entry in it:
        if entry.is_file():
            sku = int(entry.name[:-4])

            caption = images_urls_names.loc[sku].caption
            img_path = entry.path
            d_train.update({img_path:caption})
            
with os.scandir(path + 'test') as it: # TODO: cut for finetuning
    for entry in it:
        if entry.is_file():
            sku = int(entry.name[:-4]) 
            caption = images_urls_names.loc[sku].caption
            img_path = entry.path
            d_test.update({img_path:caption})

train_dataset = RuCLIPDataset(d_train, processor)
test_dataset = RuCLIPDataset(d_test, processor)
train_dataloader = DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size = BATCH_SIZE, shuffle=False)

In [20]:
broken_images = [8912890, 7402194, 24258057, 137525107, 133573560, 137525106, 7378992, 138876676, 134615956, 131486458,
 134644254, 134644249, 134101267, 158463340, 134056494, 134657405, 134056482, 193160162, 143370413, 132250661,                
 131486454, 134056490, 143540676, 134360380, 7339429, 131486455, 134101275, 134360377, 134644244, 138876732, 134101272, 
 138876735, 134644247, 138876728, 134615955, 132969388, 138876730, 138876727, 149785010, 123020319, 7709499, 6860573, 132969404,
 131029603, 138876736, 134101265, 138876682, 138876734, 131029604, 138876677, 131486453, 123013068, 134644250, 143540677, 132250663, 
 7845094, 158463341, 138876724, 123013064, 134421201, 138876725, 138876672, 138876656, 138876680, 134421208, 138876674, 134101268, 137525105, 
 138876679, 148969750, 133573556, 137525104, 138876721, 14587457, 158463348, 134421211, 9218585, 138876723, 134421205, 134101271, 15945509, 196205750, 
 134101270, 133573568, 144801057, 146821566, 134644253, 18259200, 134421217, 137525103, 132969384, 19362657, 138876731, 11991892, 18914984, 166038348, 134421204,
 138876733, 7378995, 7339432, 134615963, 138876673, 26401923] + \
[7436717, 123020322, 134101266, 138876729, 134421206, 164483662, 18884710, 35474258, 17891495, 134421216, 134101262, 25430525,    
 134101269, 9170563, 8097913, 132250662, 138876675, 158463346, 14588324, 30867705, 134101274, 123013063, 134644251, 134101273, 133573554, 138197455] + \
[158463345, 82276487, 138876720, 134056489, 138876726, 83825814]

In [23]:
path = 'D:\Programming\jupyterD\WORK\indeepa_images/2 parts/test/'
path_to_delete = list()
with os.scandir(path) as it:
    for entry in it:
        if entry.is_file():
            sku = int(entry.name[:-4])
            if sku in broken_images:
                path_to_delete.append(entry.path)

for p in path_to_delete:
    os.remove(p)

# Попытка отвязаться от либы

In [6]:
%pip install cython

   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   - -------------------------------------- 0.1/2.8 MB 991.0 kB/s eta 0:00:03
   ---- ----------------------------------- 0.3/2.8 MB 2.5 MB/s eta 0:00:02
   ---------- ----------------------------- 0.7/2.8 MB 4.5 MB/s eta 0:00:01
   ---------------- ----------------------- 1.1/2.8 MB 5.6 MB/s eta 0:00:01
   -------------------- ------------------- 1.4/2.8 MB 6.1 MB/s eta 0:00:01
   ---------------------- ----------------- 1.6/2.8 MB 5.3 MB/s eta 0:00:01
   ---------------------------------------  2.7/2.8 MB 7.9 MB/s eta 0:00:01
   ---------------------------------------  2.7/2.8 MB 7.9 MB/s eta 0:00:01
   ---------------------------------------  2.7/2.8 MB 7.9 MB/s eta 0:00:01
   ---------------------------------------  2.7/2.8 MB 7.9 MB/s eta 0:00:01
   ---------------------------------------  2.7/2.8 MB 7.9 MB/s eta 0:00:01
   ---------------------

In [1]:
!python --version

Python 3.10.12


In [10]:
%pip install more_itertools

   ---------------------------------------- 0.0/57.0 kB ? eta -:--:--
   ---------------------------------------- 57.0/57.0 kB 1.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
from concurrent.futures import ThreadPoolExecutor
from functools import partial
import io
import urllib

import PIL.Image

from datasets import load_dataset
from datasets.utils.file_utils import get_datasets_user_agent


USER_AGENT = get_datasets_user_agent()


def fetch_single_image(image_url, timeout=None, retries=0):
    for _ in range(retries + 1):
        try:
            request = urllib.request.Request(
                image_url,
                data=None,
                headers={"user-agent": USER_AGENT},
            )
            with urllib.request.urlopen(request, timeout=timeout) as req:
                image = PIL.Image.open(io.BytesIO(req.read()))
            break
        except Exception:
            image = None
    return image


def fetch_images(batch, num_threads, timeout=1, retries=0):
    fetch_single_image_with_args = partial(fetch_single_image, timeout=timeout, retries=retries)
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        batch["image"] = list(executor.map(fetch_single_image_with_args, batch["image_url"]))
    return batch


num_threads = 20
dset = load_dataset("conceptual_12m")

d:\work\market_place\Competitors\venv_conceptual\Lib\site-packages\datasets\load.py:1491: FutureWarning: The repository for conceptual_12m contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conceptual_12m
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [2]:
dset = dset.map(fetch_images, batched=True, batch_size=100, fn_kwargs={"num_threads": num_threads})

Map:   0%|          | 0/12423374 [00:00<?, ? examples/s]

KeyboardInterrupt: 